<h2>CASE STUDY 2<br>
Robin Duhan<br>
18BCS6150</h2>

**Importing the libraries**

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [2]:
#Importing the needed libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

<h4> Looking at the data and Preprocesing it </h4>

In [3]:
#Loading the data
data = pd.read_csv("/kaggle/input/titanic/train.csv")

In [4]:
#Looking at the data
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
#Shape
data.shape

(891, 12)

In [6]:
#No need of names, all are unique, so dropping it
data.drop(['Name'], axis = 1, inplace = True)

In [7]:
#Lets use a binary map for the sex column

def binary_map(x):
    if(x == "male"):
        x = 0
        return x
    elif(x == "female"):
        x = 1
        return x

data["Sex"] = data["Sex"].apply(binary_map)

In [8]:
#Unique tickets

len(data["Ticket"].unique())

681

In [9]:
# For now I am dropping this column because it seems redundant
data.drop(["Ticket"], axis = 1, inplace = True)

In [10]:
#Unique values of Cabin
data["Cabin"].unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33',
       'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101',
       'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4',
       'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35',
       'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19',
       'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54',
       'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40',
       'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44',
       'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14',
       'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38',
       'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68',
       'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48',
       'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63',
       'C62 C64',

In [11]:
# There are a lot of cabin id's so I am going to make nans as 0 and others as 1 as in if they had a cabin or not
# It could potenitally say something about their survival
# We don't need ids so removing that


def map_for_cabin(x):
    if x != 0:
        x = 1
        return x
    else:
        return 0
    

data["Cabin"] = data["Cabin"].apply(map_for_cabin)

In [12]:
data["Cabin"].head()

0    1
1    1
2    1
3    1
4    1
Name: Cabin, dtype: int64

In [13]:
#Unique values of Embarked
print("Unique values in Embarked column are:", data["Embarked"].unique())
print("Nan values are: ", data["Embarked"].isnull().sum())

#What does it mean to have nan values in this column? 
#This column represents what port you got in from
# So nan must be some data error
# As this can be important, I will be encoding it into 4 classes, one class for nan

Unique values in Embarked column are: ['S' 'C' 'Q' nan]
Nan values are:  2


In [14]:
# Adding another class N for no port
data["Embarked"].fillna("N", inplace = True)

In [15]:
Emb = pd.get_dummies(data['Embarked'])

In [16]:
# To avoid the dummy trap, dropping N
Emb.drop(["N"], axis = 1, inplace = True)
Emb.head()

,C,Q,S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1


In [17]:
# Concatenating them together

data = pd.concat([data, Emb], axis = 1)

#No use of it afte creating dummy variables
data.drop(["Embarked"], axis = 1, inplace = True)


In [18]:
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,C,Q,S
0,1,0,3,0,22.0,1,0,7.2500,1,0,0,1
1,2,1,1,1,38.0,1,0,71.2833,1,1,0,0
2,3,1,3,1,26.0,0,0,7.9250,1,0,0,1
3,4,1,1,1,35.0,1,0,53.1000,1,0,0,1
4,5,0,3,0,35.0,0,0,8.0500,1,0,0,1


In [19]:
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Cabin            0
C                0
Q                0
S                0
dtype: int64

In [20]:
#Filling the nan values with mean
data.fillna(data.mean(), inplace = True)

In [21]:
#Dropping this beacuse its redundant
data = data.drop(["PassengerId"], axis  =1)

<h3> Cleaning is done, Now splitting the Data </h3>

In [22]:
# Creating the training and validation set

np.random.seed(0)

from sklearn.model_selection import train_test_split

data_train, data_test = train_test_split(data, train_size = 0.85, test_size = 0.15, random_state = 0 )

In [23]:
#Y train for our model
y_train = data_train.pop("Survived")

In [24]:
#X train
x_train = data_train

<h2> Building our Model </h2>

In [25]:
#Building our first Model

import statsmodels.api as sm
#x_train = sm.add_constant(x_train)
model1 = sm.GLM(y_train,(sm.add_constant(x_train)), family = sm.families.Binomial()).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Survived   No. Observations:                  757
Model:                            GLM   Df Residuals:                      747
Model Family:                Binomial   Df Model:                            9
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -334.36
Date:                Sat, 29 Aug 2020   Deviance:                       668.72
Time:                        16:42:24   Pearson chi2:                     774.
No. Iterations:                    19                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Pclass        -1.0889      0.155     -7.019      0.000      -1.393      -0.785
Sex            2.6852      0.216     12.411      0.000       2.261       3.109
Age           -0.0429      0.009     -5.005      0.000      -0.060      -0.026
SibSp         -0.3259      0.118     -2.761      0.006      -0.557      -0.095
Parch         -0.0630      0.130     -0.484      0.629      -0.318       0.192
Fare           0.0013      0.002      0.556      0.578      -0.003       0.006
Cabin         21.0839   1.22e+04      0.002      0.999   -2.39e+04     2.4e+04
C            -18.4598   1.22e+04     -0.002      0.999    -2.4e+04    2.39e+04
Q            -18.4630   1.22e+04     -0.002      0.999    -2.4e+04    2.39e+04
S            -18.8457   1.22e+04     -0.002      0.999    -2.4e+04    2.39e+04
==============================================================================
"""

In [26]:
#Getting xtest and y test
y_test = data_test.pop("Survived")
x_test = data_test

In [27]:
x_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,C,Q,S
495,3,0,29.699118,0,0,14.4583,1,1,0,0
648,3,0,29.699118,0,0,7.5500,1,0,0,1
278,3,0,7.000000,4,1,29.1250,1,0,1,0
31,1,1,29.699118,1,0,146.5208,1,1,0,0
255,3,1,29.000000,0,2,15.2458,1,1,0,0


In [28]:
y_test.head()

495    0
648    0
278    0
31     1
255    1
Name: Survived, dtype: int64

In [29]:
#Using our model for prediction
x_test_sm = sm.add_constant(x_test)
y_pred = model1.predict(x_test)

In [30]:
y_pred.shape

(134,)

In [31]:
#SO as to convert it inot 0 or 1
#This is mapping for that

def Categ(x):
    if x > 0.5:
        return 1
    else:
        return 0
    
y_pred = y_pred.apply(Categ)

In [32]:
y_pred

495    0
648    0
278    0
31     1
255    1
      ..
150    0
501    1
494    0
784    0
113    1
Length: 134, dtype: int64

**Model Evaluation on validating data**

In [33]:
from sklearn import metrics

In [34]:
#Accuracy is good 
print(metrics.accuracy_score(y_pred, y_test))

0.7910447761194029


In [35]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif['Features'] = x_train.columns
vif['VIF'] = [variance_inflation_factor(x_train.values, i) for i in range(x_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

,Features,VIF
6,Cabin,418.70
9,S,77.27
7,C,58.23
8,Q,32.45
0,Pclass,1.75
5,Fare,1.61
4,Parch,1.33
3,SibSp,1.28
2,Age,1.21
1,Sex,1.14


In [36]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,C,Q,S
0,0,3,0,22.0,1,0,7.2500,1,0,0,1
1,1,1,1,38.0,1,0,71.2833,1,1,0,0
2,1,3,1,26.0,0,0,7.9250,1,0,0,1
3,1,1,1,35.0,1,0,53.1000,1,0,0,1
4,0,3,0,35.0,0,0,8.0500,1,0,0,1


In [37]:
#removing these columns ["Cabin", "C", "S", "Q"] and building the model again
col = ["Cabin", "C", "S", "Q"]

In [38]:
data_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,C,Q,S
495,3,0,29.699118,0,0,14.4583,1,1,0,0
648,3,0,29.699118,0,0,7.5500,1,0,0,1
278,3,0,7.000000,4,1,29.1250,1,0,1,0
31,1,1,29.699118,1,0,146.5208,1,1,0,0
255,3,1,29.000000,0,2,15.2458,1,1,0,0


<h3> RFE and Building our Model Again </h3>

In [39]:
#RFE to get the top 5 features and using them to build our model

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(max_iter=400)

from sklearn.feature_selection import RFE
rfe = RFE(logreg, 5) # running RFE with 13 variables as output
rfe = rfe.fit(x_train, y_train)


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_features_to_select=5 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [40]:
col1 = x_train.columns[rfe.support_]
col1

Index(['Pclass', 'Sex', 'SibSp', 'C', 'S'], dtype='object')

In [41]:
#Training the model
x_train_sm = sm.add_constant(x_train[col1])
model2 = sm.GLM(y_train,x_train_sm, family = sm.families.Binomial())
model2 = model2.fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Survived   No. Observations:                  757
Model:                            GLM   Df Residuals:                      751
Model Family:                Binomial   Df Model:                            5
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -348.49
Date:                Sat, 29 Aug 2020   Deviance:                       696.98
Time:                        16:42:25   Pearson chi2:                     769.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6838      0.455      1.501      0.133      -0.209       1.576
Pclass        -0.8586      0.119     -7.188      0.000      -1.093      -0.624
Sex            2.6943      0.207     13.002      0.000       2.288       3.100
SibSp         -0.2356      0.103     -2.295      0.022      -0.437      -0.034
C              0.1831      0.389      0.470      0.638      -0.580       0.946
S             -0.2781      0.336     -0.828      0.408      -0.937       0.380
==============================================================================
"""

In [42]:
x_test[col1].head()

,Pclass,Sex,SibSp,C,S
495,3,0,0,1,0
648,3,0,0,0,1
278,3,0,4,0,0
31,1,1,1,1,0
255,3,1,0,1,0


In [43]:
#Prediction
x_test_t = sm.add_constant(x_test[col1])
y_pred1 = model2.predict(x_test_t)

In [44]:
y_pred1 = y_pred1.apply(Categ)

In [45]:
y_pred1

495    0
648    0
278    0
31     1
255    1
      ..
150    0
501    1
494    0
784    0
113    1
Length: 134, dtype: int64

In [46]:
print(metrics.accuracy_score(y_pred1, y_test))

0.8059701492537313


In [47]:
#VIF
#VIF is good enough
x_train_t = x_train[col1]
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif['Features'] = col1
vif['VIF'] = [variance_inflation_factor(x_train_t.values, i) for i in range(x_train_t.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

,Features,VIF
0,Pclass,4.56
4,S,4.03
3,C,1.58
1,Sex,1.49
2,SibSp,1.26


<h2> Preparing test data for prediction and kaggle submission </h2>

In [48]:
test = pd.read_csv("/kaggle/input/titanic/test.csv")
test.shape

(418, 11)

In [49]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [50]:
test.drop(["Name", "PassengerId"], axis = 1, inplace = True)

In [51]:
test["Sex"] = test["Sex"].apply(binary_map)

In [52]:
test.drop(["Ticket"], axis = 1, inplace = True)

In [53]:
test["Cabin"] = test["Cabin"].apply(map_for_cabin)

In [54]:

Emb1 = pd.get_dummies(test['Embarked'])
test = pd.concat([test, Emb1], axis = 1)

#No use of it afte creating dummy variables
test.drop(["Embarked"], axis = 1, inplace = True)


In [55]:
test = test[col1]

In [56]:
y_pred2 = model2.predict(sm.add_constant(test))

In [57]:
y_pred2 = y_pred2.apply(Categ)

In [58]:
y_pred2

0      0
1      1
2      0
3      0
4      1
      ..
413    0
414    1
415    0
416    0
417    0
Length: 418, dtype: int64

<h3> Kaggle submission </h3>

In [59]:
sub = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")
my_submission = pd.DataFrame({'PassengerId':sub.PassengerId,'Survived': y_pred2})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)